In [1]:
import pandas as pd
import numpy as np

In [2]:
food = pd.read_csv("/Users/anjali6810/Desktop/Food_Menu.csv", encoding = 'utf8')

In [3]:
food_list = food.Food_Name

In [4]:
food.head()

,S_No,Food_Name,Ingredients,Veg/Non-veg,Course
0,1,Poha,"Beaten rice flakes, potato, curry leaves, gree...",vegetarian,snack
1,2,Fara,"Chana dal, whole wheat flour, arhar dal, white...",vegetarian,snack
2,3,Kachori,"Moong dal, rava, garam masala, dough, fennel s...",vegetarian,snack
3,4,Pani puri,"Kala chana, mashed potato, boondi, sev, lemon",vegetarian,snack
4,5,Papad,"Urad dal, sev, lemon juice, chopped tomatoes",vegetarian,snack


In [5]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   S_No         316 non-null    int64 
 1   Food_Name    316 non-null    object
 2   Ingredients  316 non-null    object
 3   Veg/Non-veg  316 non-null    object
 4   Course       316 non-null    object
dtypes: int64(1), object(4)
memory usage: 12.5+ KB


In [6]:
food.isnull().sum()

S_No           0
Food_Name      0
Ingredients    0
Veg/Non-veg    0
Course         0
dtype: int64

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(stop_words = "english")

tfidf_matrix = tfidf.fit_transform(food.Ingredients)
tfidf_matrix.shape

(316, 334)

In [8]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
food_index = pd.Series(food.index, index = food['Food_Name']).drop_duplicates()
food_index
food_id = food_index["Masala Dosa"]
food_id

18

In [10]:
def get_recommendations(Food, topN):    
    food_id = food_index[Food]
    cosine_scores = list(enumerate(cosine_sim_matrix[food_id]))
    cosine_scores = sorted(cosine_scores, key=lambda x:x[1], reverse = True)
    cosine_scores_N = cosine_scores[0: topN+1]
    
    food_idx  =  [i[0] for i in cosine_scores_N]
    food_scores =  [i[1] for i in cosine_scores_N]

    food_similar_show = pd.DataFrame(columns=["Food_Name", "Score"])
    food_similar_show["Food_Name"] = food.loc[food_idx, "Food_Name"]
    food_similar_show["Score"] = food_scores
    food_similar_show.reset_index(inplace = True)  
    print (food_similar_show)

In [11]:
get_recommendations("Dosa", topN = 10)

    index           Food_Name     Score
0      12                Dosa  1.000000
1      13           Neer dosa  0.960567
2      15                Idli  0.708944
3     125             Poriyal  0.578623
4      29             Uttapam  0.578165
5      18         Masala Dosa  0.571078
6      22  Mysore Masala Dosa  0.571078
7      31                Upma  0.567222
8     113        Keerai kootu  0.565888
9     117               Kootu  0.549945
10     20  Cheese Masala Dosa  0.548457


In [12]:
import pickle

pickle.dump(food_list,open('food_list.pkl','wb'))
pickle.dump(cosine_sim_matrix,open('similarity.pkl','wb'))